In [2]:
from __future__ import division
from numba import cuda
import numpy
import math
# CUDA kernel
@cuda.jit
def degreeDist(A, C):
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k]
        C[row, col] = tmp
        
# Host code

nRows = 100; 
nCols = 100;

# Initialize the data arrays
A = np.matrix(np.loadtxt('adj.csv'))

# Copy the arrays to the device
A_global_mem = cuda.to_device(A)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((A.shape[0],1))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(A.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel
%timeit -o degreeDist[blockspergrid, threadsperblock](A_global_mem, C_global_mem)

# Copy the result back to the host
C = C_global_mem.copy_to_host()


NameError: name 'np' is not defined

In [18]:
#import constant_vars as const
import numpy as np

# NUMBER OF KERNELS TO BE USED : DEFAULT VALUE = 16
k_num = 16;
file_name = 'adj.csv';

# READ ADJACENCY MATRIX
matrix = np.full((100000, 100000), 1, numpy.float);

# DIMENSIONS OF ROWS AND COLUMNS
nRows = matrix.shape[0];
nCols = matrix.shape[1];
        
class matArr:
    def __init__(self):
        self.matrix=[];
    def addRow(self,row):
        self.matrix.append(row);
    def getMat(self):
        return self.matrix;
    def getShape(self, axis):
        return np.matrix(self.matrix).shape[axis];

class genMatrices:
    
    # GET GENERATED ARRAY OF MATRICES
    def getMatArray(self):
        # DEFINE NUMBER OF SPLITS ON THE ADJACENCY MATRIX
        rowsPerSplit = nRows//k_num;
        adRows = nRows-(rowsPerSplit*k_num);

        # Define empty matrices for K_NUM kernels
        matList = [matArr() for i in range(k_num)];

        # Populate Matrices with rowsPerSplit number of rows
        counter = 0;
        for matNum in range(k_num):
            for _ in range(rowsPerSplit):
                if(counter<nRows):
                    matList[matNum].addRow(matrix[counter]);
                    counter+=1;

        # Populate remaining rows into the last matrix           
        for i in range(counter,nRows):
            matList[k_num-1].addRow(matrix[i]);
            
        return matList;
        
genMat = genMatrices();
arr=genMat.getMatArray();

MemoryError: 

In [19]:
from __future__ import division
from numba import cuda
import numpy as np
import math
# CUDA kernel
@cuda.jit
def degreeDist(A, C):
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k]
        C[row, col] = tmp
        
# Host code


# Start the kernel
def testPar():
    for i in range(len(arr)):

        # Copy the arrays to the device
        A_global_mem = cuda.to_device((arr[i]))

        # Allocate memory on the device for the result
        C_global_mem = cuda.device_array((arr[i].getShape(0),1))

        # Configure the blocks
        threadsperblock = (16, 16)
        blockspergrid_x = int(math.ceil(arr[i].getShape(0) / threadsperblock[0]))
        blockspergrid_y = int(math.ceil(arr[i].getShape(1) / threadsperblock[1]))
        blockspergrid = (blockspergrid_x, blockspergrid_y)

        degreeDist[blockspergrid, threadsperblock](np.matrix(arr[i].getMat()), C_global_mem)
        cuda.synchronize()
        # Copy the result back to the host
        C = C_global_mem.copy_to_host()
%timeit testPar()


2.56 s ± 17.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
